In [81]:
from __future__ import absolute_import, print_function
import csv

In [82]:
import json, os, re


In [84]:
identifier_fields = {
    "dwc:Occurrence": [
        ("idigbio:recordId", lambda r, rs: mungeid(r)),
        ("idigbio:recordID", lambda r, rs: mungeid(r)),
        ("dwc:ResourceRelationship", lambda r, rs: idFromRR(r, rs=rs)),
        ("dwc:occurrenceID", lambda r, rs: rs + "\\" + mungeid(r)),
        ("id", lambda r, rs: rs + "\\" + mungeid(r)),
        ("ID", lambda r, rs: rs + "\\" + mungeid(r)),
    ],
    "dwc:Multimedia": [
        ("idigbio:recordId", lambda r, rs: mungeid(r)),
        ("idigbio:recordID", lambda r, rs: mungeid(r)),
        ("ac:providerManagedID", lambda r, rs: mungeid(r)),
        ("dcterms:identifier", lambda r, rs: rs + "\\media\\" + mungeid(r)),
        # ("coreid", lambda r,rs: rs + "\\" + r)
    ],
    "dcterms": [
        ("idigbio:recordId", lambda r, rs: mungeid(r)),
        ("idigbio:recordID", lambda r, rs: mungeid(r)),
        ("ac:providerManagedID", lambda r, rs: mungeid(r)),
        ("dcterms:identifier", lambda r, rs: rs + "\\media\\" + mungeid(r)),
        # ("coreid", lambda r,rs: rs + "\\" + r)
    ]
}


In [85]:
def identifyRecord(t, etag, r, rsid):
    idents = []
    if t in identifier_fields:
        for pi in identifier_fields[t]:
            if pi[0] in r:
                # The "UConn" exception
                if pi[0] == "ac:providerManagedID" and "dcterms:identifier" in r and r["dcterms:identifier"].lower() == r["ac:providerManagedID"].lower():
                    continue
                cid = pi[1](r[pi[0]], rsid)
                if cid is not None:
                    idents.append((etag, pi[0], cid.lower()))
                    # Breaking would give us only the first ID
                    # break
    return idents


In [86]:
def mungeid(s):
    return bad_char_re.sub('', s).strip()


In [88]:
import idigbio
idigbio.__version__
api = idigbio.json()

In [131]:
missing_api_query = {"data.ac:providerManagedID": {"type":"missing"}}
present_api_query = {"data.ac:providerManagedID": {"type":"exists"}}
media_identifier_keys = ["dcterms:identifier", "idigbio:recordId", "ac:providerManagedID"]

Media missing ac:providerManagedID
======================================

In [129]:
media_missing_ac_providerManagedID = api.search_media(mq=missing_api_query, fields=["uuid","data.dcterms:identifier","data.idigbio:recordId"])
if 'iteritems' in dir(media_present_ac_providerManagedID):
    print ("OK")
else:
    print ("QUERY FAILED")

OK


In [93]:
media_missing_ac_providerManagedID.keys()

[u'lastModified', u'itemCount', u'attribution', u'items']

In [94]:
media_missing_ac_providerManagedID['items']

[{u'data': {u'idigbio:recordId': u'TTD_TCN_COLO_01608710.jpg'},
  u'indexTerms': {u'uuid': u'1b648fc5-dbca-4924-b902-7cf6dd1fb8ca'},
  u'type': u'mediarecords',
  u'uuid': u'1b648fc5-dbca-4924-b902-7cf6dd1fb8ca'},
 {u'data': {u'idigbio:recordId': u'TTD_TCN_COLO_01067271.jpg'},
  u'indexTerms': {u'uuid': u'1b64e8ca-10cb-4447-8ab8-663b54b6183a'},
  u'type': u'mediarecords',
  u'uuid': u'1b64e8ca-10cb-4447-8ab8-663b54b6183a'},
 {u'data': {u'idigbio:recordId': u'TTD_TCN_MICH_1119829.jpg'},
  u'indexTerms': {u'uuid': u'1b64fe32-dd51-4205-a655-5e8d8be9ddc8'},
  u'type': u'mediarecords',
  u'uuid': u'1b64fe32-dd51-4205-a655-5e8d8be9ddc8'},
 {u'data': {u'idigbio:recordId': u'TTD_TCN_MU_000075720.jpg'},
  u'indexTerms': {u'uuid': u'1b650b6d-dbd5-4254-80d9-ef8315adebaa'},
  u'type': u'mediarecords',
  u'uuid': u'1b650b6d-dbd5-4254-80d9-ef8315adebaa'},
 {u'data': {u'idigbio:recordId': u'TTD_TCN_MICH_1402577.jpg'},
  u'indexTerms': {u'uuid': u'1b65d6a5-5d51-49d9-928e-b81624b26a76'},
  u'type': u'm

Media containing ac:providerManagedID
======================================

In [132]:
##################################################################

media_present_ac_providerManagedID = api.search_media(mq=present_api_query, fields=["uuid","data.dcterms:identifier","data.idigbio:recordId", "data.ac:providerManagedID"], limit=10)
if 'iteritems' in dir(media_present_ac_providerManagedID):
    print ("OK")
else:
    print ("QUERY FAILED")

OK


In [306]:
presents = {}
for each in media_present_ac_providerManagedID['items']:
    presents[each["uuid"]] = []
    for key in media_identifier_keys:
        if key in (each['data']):
            #print (key, each['data'][key])
            presents[each["uuid"]].append(each['data'][key]) #each['data'][key]]

    #media_present_ac_providerManagedID['items']
    #key = media_missing_ac_providerManagedID['items'][key]
    #print (key)
#.keys():
    
    #print (each)
    #print (media_missing_ac_providerManagedID['items'][0][each])
    #if each['data'].keys() != 'idigbio:recordId':
    #    print each['data'].keys()

In [314]:
i = 0
for some in presents:
    print(some,presents[some])
    i+=1
    if i > 10:
        break

    

1bf05878-e7ae-4acc-972c-59451f9eae5a [u'urn:uuid:89338acb-48a0-44b3-8e68-99f6895b6b80', u'http://www.morphbank.net/77711']
1bdf34c8-25c7-4c4d-9c03-d01ce172b7d3 [u'urn:uuid:bc096091-e804-487c-8285-e82356f63a28', u'http://www.morphbank.net/669193']
1bbd1232-99f5-4f1d-bab3-c30400ca5e86 [u'urn:uuid:91562f1c-d7c0-48d9-95ef-23655258655e', u'http://www.morphbank.net/99751']
1ba42ac4-5f33-4396-a1f8-489450684597 [u'urn:uuid:63ec2561-1ef0-4984-8ff7-85a2bbfa725a', u'http://www.morphbank.net/718258']
1bc60038-72c6-447d-81f2-974583cdf827 [u'urn:uuid:46fe955a-9a4a-4051-a921-fac3e629b09f', u'http://www.morphbank.net/746085']
1bf44040-a4d4-491a-8a52-62b04a648795 [u'urn:uuid:5f3dd9dd-a725-4132-9b67-d50d8741b34e', u'http://www.morphbank.net/592186']
1bd865fa-0b8c-453c-bf2b-75b53c2d1623 [u'urn:uuid:818beed1-8ca4-4bcc-b7ec-3d6464d2eb3e', u'http://www.morphbank.net/754647']
1c0b3a87-dbca-4eb8-8594-cd06b8334df0 [u'urn:uuid:e81e93a0-560c-4788-8423-a5d55f232d0d', u'http://www.morphbank.net/709686']
1bd9a43a-6

In [109]:
matched = {}
for item in media_present_ac_providerManagedID['items']:
        u = item["uuid"]
        i = item["data"]["dcterms:identifier"]
        matched[u] = i

for each in matched:
    print(matched)

{u'1d183f2b-ab82-4ecf-b521-51d82d14ed38': u'urn:uuid:b31783c9-a54d-4a37-8a5e-605c68c09c95', u'1f2dbb2b-75ba-48cb-b34c-1ca003b4a38d': u'urn:uuid:9fdcf8c8-5c90-408e-ae97-a6d8579cf63f', u'1ba42ac4-5f33-4396-a1f8-489450684597': u'urn:uuid:63ec2561-1ef0-4984-8ff7-85a2bbfa725a', u'1f5b2fbe-0001-4247-8943-cc940ee16d81': u'urn:uuid:fb68fbe3-3c70-4039-a2b6-e86e187384f3', u'1f1546f3-2008-4076-8200-ebaf02cc2f65': u'urn:uuid:6d9d7aff-c861-4b28-a388-d0a416f52ddc', u'1bd865fa-0b8c-453c-bf2b-75b53c2d1623': u'urn:uuid:818beed1-8ca4-4bcc-b7ec-3d6464d2eb3e', u'1f11e6d9-fa89-4578-bea0-c9f0c60c10ff': u'urn:uuid:03594ac5-6ad9-4ad2-b947-f44ca8edc77b', u'1c7ce211-cec3-4f15-a179-f17dbffeb894': u'urn:uuid:2e33421c-2bed-45a8-a239-e6ed90b3c178', u'1cde90b8-56c3-4d89-a0ad-c2908c684220': u'urn:uuid:8b8469b4-151d-42ef-a816-357d81b6417b', u'1e1aeb79-4a17-4075-9fde-a9a05d5fc802': u'urn:uuid:dd4c420f-30bc-47aa-aeba-a9d11d9f90be', u'1d794547-8228-42d6-8f3d-9ad39b2b981f': u'urn:uuid:214c99b9-5168-45dd-8941-26dd0eac0c20'

uuids with multiple identifiers (in database)
==============================================

In [1]:
from psycopg2 import DatabaseError
from psycopg2.extras import DictCursor

from idb.postgres_backend.db import PostgresDB, RecordSet


In [2]:
db = PostgresDB()

In [3]:
throw_away_uuids = set()
keep_uuids = set()

In [6]:
number_sql = """select uuids_id from uuids_identifier where identifier ~ E'^\\\d+$'"""
uuids_ids_with_numbers_identifier = db.fetchall(number_sql)
len(uuids_ids_with_numbers_identifier)

0

In [348]:
number_identifiers = {}
#db.rollback()
for each in uuids_ids_with_numbers_identifier:
#    print (type(each))
    # each["uuids_id"] is the string uuids
    uuid = each["uuids_id"]
#    print (uuid)
    each_sql = "select identifier from uuids_identifier where uuids_id = '{0}'".format(uuid)
#    print (each_sql)
    resultz = db.fetchall(each_sql)
#    print (len(resultz))
    number_identifiers[uuid] = resultz
    #["identifier"]
    
print (len(number_identifiers))

681238


In [350]:
# keep a set of these uuids so we do not query for them again
for each in uuids_ids_with_numbers_identifier:
    throw_away_uuids.add(each["uuids_id"])
len(throw_away_uuids)

681238

In [353]:
row_count = 0
with open('all_identifiers_with_providerManagedID.csv', 'wb') as csv_file:
    #writer = csv.writer(csv_file)
    for key, value in number_identifiers.items():
        row = ""
        remaining_list = []
        #remaining_list = [key]
        #row += key + ","
        #print(row)
        for each in value:
            remaining_list.append(each[0])

        #','.join(row,remaining)
        #print(remaining_list)
        remaining_list.sort(key=len)
        row_end = '\t'.join(remaining_list)
        row = key + "\t" + row_end + "\n"
        #print(row)
        #break
        csv_file.write(row)
        row_count += 1
row_count

681238

needs the set generated above!


In [359]:
all_media_uuids_sql = """select uuids_id from uuids_identifier, uuids where uuids.type = 'mediarecord' and uuids.id = uuids_identifier.uuids_id and uuids_identifier.identifier !~ E'^\\d+$' order by uuids_id limit 100000"""
uuids_ids_others = db.fetchall(all_media_uuids_sql)
len(uuids_ids_others)


100000

In [360]:
media_identifiers = {}

for each in uuids_ids_others:
    uuid = each["uuids_id"]
    if uuid not in throw_away_uuids:
        keep_uuids.add(uuid)
        each_sql = "select identifier from uuids_identifier where uuids_id = '{0}'".format(uuid)
        resultz = db.fetchall(each_sql)
        media_identifiers[uuid] = resultz
   
print (len(media_identifiers))
print (len(keep_uuids))

72206
72206


In [366]:
row_count = 0
with open('media_identifiers_non-numbers.csv', 'wb') as csv_file:
    for key, value in media_identifiers.items():
        row = ""
        remaining_list = []
        for each in value:
            remaining_list.append(each[0])

        remaining_list.sort(key=len)
        row_end = '\t'.join(remaining_list)
        row = key + "\t" + row_end + "\t \t \t \t \n"
        try:
            csv_file.write(row)
        except:
            print(repr(row))
        row_count += 1
row_count


u'004dd4c6-ea21-4420-8c02-7fd34dfd89d4\t616857b7-f952-44ef-9b6f-576dc1e65b51\\media\\http://dsiphoto.mnhn.fr/sonnerat/pr\xeat-phan\xe9ro/prt-jpg-rtrait\xe9-003/p00334130.jpg\t \t \t \t \n'
u'00654433-d4bd-4ea6-9637-8d06e3afe037\t616857b7-f952-44ef-9b6f-576dc1e65b51\\media\\http://dsiphoto.mnhn.fr/sonnerat/gbif/sol-orc-jpeg-050 \xe0 v\xe9rif/p00418971.jpg\t \t \t \t \n'


72206

In [301]:
print(len(r'205fa34c-2fcb-4492-b992-972b18560f6f\media\48d40212-ee97-4d24-aadb-95052450dfeb'))

print(len(r'c481fbc6-4bd7-4c50-8537-ba1993d4eb88\media\34e0f7c2-b1c2-429e-a6fd-ac0b91da1dd8'))

print(len(r'bd7cfd55-bf55-46fc-878d-e6e11f574ccd\media\ark:/87602/m4/m11379'))

print(len(r'32d433aa-9e2b-4ff9-bc55-5c3e30112207\media\0968f392-d91a-440c-be8f-3466d051f09b'))


79
79
63
79
